In [1]:
# gather stats for the release

In [2]:
import pandas as pd
import numpy as np
import subprocess
import random
from scipy import stats
import pickle
import glob
import math
import csv
import sys
import os
import re

import matplotlib.pyplot as plt
from matplotlib import animation
import seaborn as sns
plt.rcParams['figure.figsize'] = (20.0, 10.0)
plt.rcParams['font.family'] = "serif"
plt.rcParams['font.size'] = 24
%matplotlib inline

In [3]:
pd.set_option('display.max_columns', None)

%load_ext autoreload
%autoreload 1

sys.path.insert(0, "./soft")
%aimport definitions

In [4]:
base_dir = "./"
data_dir = base_dir+"data/"
chess3_data_dir = data_dir+"chess3/"

gff3cols = ["seqid","source","type","start","end","score","strand","phase","attributes"]

In [6]:
chess30_gtf_fname = chess3_data_dir+"chess3.0.gtf"
chess301_gtf_fname = base_dir+"chess3.0.1/"+"chess3.0.1.gtf"

In [47]:
c3df = definitions.get_attribute(chess301_gtf_fname,["gene_id","gene_type"],[0])
c3df.columns = ["tid","seqid","gid","gene_type"]
c3df.replace({'gene_type': {"C_region":"other",
                            "C_region_pseudogene":"pseudogene",
                            "D_segment":"other",
                            "J_segment":"other",
                            "J_segment_pseudogene":"pseudogene",
                            "RNase_MRP_RNA":"other",
                            "RNase_P_RNA":"other",
                            "TEC":"other",
                            "V_segment":"other",
                            "V_segment_pseudogene":"pseudogene",
                            "Y_RNA":"other",
                            "antisense_RNA":"other",
                            "lncRNA":"lncRNA",
                            "miRNA":"other",
                            "misc_RNA":"other",
                            "ncRNA":"other",
                            "ncRNA_pseudogene":"pseudogene",
                            "other":"other",
                            "protein_coding":"protein_coding",
                            "pseudogene":"pseudogene",
                            "rRNA":"other",
                            "scRNA":"other",
                            "snRNA":"other",
                            "snoRNA":"other",
                            "tRNA":"other",
                            "telomerase_RNA":"other",
                            "transcribed_pseudogene":"pseudogene",
                            "vault_RNA":"other"}},inplace=True)
c3df_pri = c3df[~(c3df["seqid"].str.contains("_alt"))].reset_index(drop=True)[["tid","gid","gene_type"]]
c3df_alt = c3df[c3df["seqid"].str.contains("_alt")].reset_index(drop=True)
c3df.head()

,tid,seqid,gid,gene_type
0,CHS.1.1,chr1,CHS.1,pseudogene
1,CHS.2.1,chr1,CHS.2,pseudogene
2,CHS.166734.2,chr1,CHS.166734,other
3,CHS.166734.1,chr1,CHS.166734,other
4,CHS.166734.3,chr1,CHS.166734,other


In [48]:
print("number of transcripts on Alternative scaffolds: "+str(len(c3df_alt)))
print("number of genes on Alternative scaffolds: "+str(len(set(c3df_alt["gid"]))))

number of transcripts on Alternative scaffolds: 10088
number of genes on Alternative scaffolds: 5250


In [49]:
print("transcripts")
grp = c3df_pri[["tid","gene_type"]].groupby(by="gene_type").count().reset_index()
grp

transcripts


,gene_type,tid
0,lncRNA,34709
1,other,7190
2,protein_coding,99201
3,pseudogene,17263


In [50]:
# genes
print("genes")
c3df_pri_gene = c3df_pri[["gid","gene_type"]].drop_duplicates()
grp = c3df_pri_gene[["gid","gene_type"]].groupby(by="gene_type").count().reset_index()
grp

genes


,gene_type,gid
0,lncRNA,17624
1,other,4269
2,protein_coding,19838
3,pseudogene,16774
